# 1. Importando os dados:

Abaixo, usamos pandas para importar os dados, mas logo transformamos o DataFrame em uma lista, para tentar nos adequar ao máximo ao conteúdo já ministrado.

In [40]:
import pandas as pd

dados = pd.read_json('brasileirao-2019.json')


campeonato = []

for i in dados.columns:
  campeonato.append(list(dados[i]))

# 2. Funções usadas:
Abaixo, criamos as funções que irão ser usadas ao longo do projeto:

In [27]:
def tecnicos(campeonato: list) -> list:
  """retorna a lista de técnicos que participaram do campeonato"""
  tec = sorted(
      list(
          set(
              [
                  tecnico
                  for rodada in campeonato
                  for partida in rodada
                  for tecnico in partida['coach'].values()
              ]
            )
          )
      )
  return tec



def times(campeonato: list) -> list:
  """retorna uma lista com os times que disputaram o campeonato em ordem alfabética"""
  t = sorted(
      list(
          set(
              [
                  time
                  for rodada in campeonato
                  for partida in rodada
                  for time in partida['clubs'].values()
              ]
            )
          )
      )
  return t



def times_por_tecnicos(tecnico: str, campeonato: list) -> dict:
  """Retorna um dicionário, as chaves são times,
  e os valores a quantidade de partidas que o técnico esteve no time"""
  times = [
    partida['clubs'][chave]
    for rodada in campeonato
    for partida in rodada
    for chave in ['home', 'away']
    if partida['coach'][chave] == tecnico
]
  return {time:times.count(time) for time in times}


def chave_tecnico(tecnico: str, partida: dict) -> str:
  """Recebe um tecnico e uma partida, e retorna a chave
  que indica se o tecnico jogou em casa ou fora de casa"""
  if tecnico not in partida['coach'].values():
    return ""
  return [ch for ch in partida['coach'].keys() if partida['coach'][ch] == tecnico][0]



def chave_time(time: str, partida: dict) -> str:
  """Recebe um time e uma partida, e retorna a chave
  que indica se o time jogou em casa ou fora de casa"""
  if time not in partida['clubs'].values():
    return ""
  return [ch for ch in partida['clubs'].keys() if partida['clubs'][ch] == time][0]



def tecnico_de(time: str, partida: dict) -> str:
  """Retorna o ténico do time na partida"""
  return "" if time not in partida['clubs'].values() else partida['coach'][chave_time(time, partida)]



def time_de(tecnico: str, partida: dict) -> str:
    """retorna o time treinado por um tecnico em uma partida"""
    return "" if tecnico not in partida['coach'].values() else partida['clubs'][chave_tecnico(tecnico, partida)]



def vencedor(partida: dict) -> str:
  """recebe uma partida e retorna o time vencedor"""
  if partida['goals']['home'] == partida['goals']['away']:
    return "empate"
  else:
    home = int(partida['goals']['home'])
    away = int(partida['goals']['away'])
    return partida['clubs']['home'] if home > away else partida['clubs']['away']



def pontos(time: str, campeonato: list) -> int:
  """retorna a quantidade de pontos de um time em um campeonato"""
  pontos = 0
  partidas = [
      partida
      for rodada in campeonato
      for partida in rodada
      if time in partida['clubs'].values()
    ]
  for partida in partidas:
    if vencedor(partida) == time:
      pontos += 3
    elif vencedor(partida) == 'empate':
      pontos += 1
  return pontos



def pontos_tecnico(tecnico: str, time: str, campeonato: list) -> int:
    """Retorna a quantidade de pontos que um técnico conseguiu para um dado time em um campeonato"""
    pontos = 0
    for rodada in campeonato:
        for partida in rodada:
            if time in partida['clubs'].values() and tecnico_de(time, partida) == tecnico:
                if vencedor(partida) == time:
                    pontos += 3
                elif vencedor(partida) == 'empate':
                    pontos += 1
    return pontos

# 3. Sei lá


70